In [2]:
import pandas as pd
import numpy as np

# Metrics for regression evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Preprocessing tools
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Main ML model
import xgboost as xgb

# Classical time-series models
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet

import warnings
warnings.filterwarnings("ignore")


In [15]:
# Load your dataset
df = pd.read_csv("widr_zone_day_model_ready_2021_2023.csv")

# Convert date column to datetime and sort by zone and time
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["zone_id", "date"]).reset_index(drop=True)

# Ensure target exists
target = "waste_tons_day"

print("Dataset shape:", df.shape)
print("Columns:", df.columns)
# Create time-based split

# Choose split dates
train_end = "2022-12-31"
val_end   = "2023-06-30"

train = df[df["date"] <= train_end].copy()
val   = df[(df["date"] > train_end) & (df["date"] <= val_end)].copy()
test  = df[df["date"] > val_end].copy()

print("Train shape:", train.shape)
print("Val shape:", val.shape)
print("Test shape:", test.shape)

# Safety check
print("Train max date:", train["date"].max())
print("Test min date:", test["date"].min())


Dataset shape: (37345, 41)
Columns: Index(['date', 'zone_id', 'zone_name', 'population', 'waste_baseline_tons',
       'dow', 'month', 'is_umuganda', 'event_count_cal', 'event_intensity_cal',
       'rain_mm', 'temp_c', 'humidity', 'ndvi', 'season_month', 'season_week',
       'event_factor', 'weather_factor', 'ndvi_factor', 'waste_tons_day',
       'year', 'day', 'is_weekend', 'weekofyear', 'month_sin', 'month_cos',
       'dow_sin', 'dow_cos', 'waste_tons_day_lag1', 'waste_tons_day_lag7',
       'waste_tons_day_lag14', 'waste_tons_day_lag28',
       'waste_tons_day_roll7_mean', 'waste_tons_day_roll14_mean',
       'waste_tons_day_roll28_mean', 'ndvi_lag1', 'ndvi_lag7', 'ndvi_lag14',
       'ndvi_roll7_mean', 'ndvi_roll14_mean', 'ndvi_roll28_mean'],
      dtype='object')
Train shape: (24570, 41)
Val shape: (6335, 41)
Test shape: (6440, 41)
Train max date: 2022-12-31 00:00:00
Test min date: 2023-07-01 00:00:00


In [16]:

def safe_metrics(y_true, y_pred):

    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    # Only evaluate where both true and predicted values are finite
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    if mask.sum() == 0:
        return np.nan, np.nan

    mae = mean_absolute_error(y_true[mask], y_pred[mask])
    rmse = np.sqrt(mean_squared_error(y_true[mask], y_pred[mask]))
    return mae, rmse


def eval_naive_lag1(test_df, target_col, lag1_col="waste_tons_day_lag1"):
    """
    Naive baseline model:
    Predict today's waste using yesterday's waste (Lag-1).
    This is a strong baseline for time-series forecasting.
    """
    if lag1_col not in test_df.columns:
        raise ValueError(f"Missing '{lag1_col}'. Create lag features first.")

    return test_df[lag1_col].values


def train_xgb_forecaster(train_df, test_df, target_col):
    """
    Train an XGBoost regression model for waste forecasting.

    Model pipeline:
    - One-hot encode zone_id (categorical spatial identifier)
    - Pass all numeric engineered features unchanged
    - Train gradient-boosted regression trees
    """

    # Remove non-feature columns
    drop_cols = ["date", "zone_name", target_col]
    drop_cols = [c for c in drop_cols if c in train_df.columns]

    features = [c for c in train_df.columns if c not in drop_cols]

    # Separate categorical and numeric inputs
    categorical_cols = ["zone_id"]
    numeric_cols = [c for c in features if c not in categorical_cols]

    # Preprocessing: encode spatial ID, keep numeric signals
    preprocessor = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ])

    # XGBoost regression model (main forecasting engine)
    xgb_model = xgb.XGBRegressor(
        n_estimators=1200,
        learning_rate=0.03,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        objective="reg:squarederror",
        random_state=42
    )

    # Combine preprocessing + model into a single pipeline
    model = Pipeline([
        ("prep", preprocessor),
        ("model", xgb_model)
    ])

    # Train model
    X_train, y_train = train_df[features], train_df[target_col]
    X_test = test_df[features]

    model.fit(X_train, y_train)

    # Generate predictions for evaluation period
    pred = model.predict(X_test)

    return pred, model


In [17]:
# Classical baselines: ARIMA and Prophet

def arima_forecast_per_zone(train_df, test_df, target_col,
                            zone_col="zone_id", date_col="date",
                            order=(2, 1, 2)):
    """
    ARIMA forecasting baseline applied independently per zone.
    Captures temporal autocorrelation but ignores exogenous/context features.
    """
    out_rows = []
    zones = sorted(test_df[zone_col].unique())

    for z in zones:
        tr = train_df[train_df[zone_col] == z].sort_values(date_col)
        te = test_df[test_df[zone_col] == z].sort_values(date_col)

        # Skip zones without enough history for ARIMA fitting
        if len(tr) < 60 or len(te) == 0:
            continue

        # Fit ARIMA on historical target signal
        fit = ARIMA(tr[target_col].values, order=order).fit()
        fc = fit.forecast(steps=len(te))

        out_rows.append(pd.DataFrame({
            zone_col: z,
            date_col: te[date_col].values,
            "pred_arima": fc
        }))

    if not out_rows:
        return pd.DataFrame(columns=[zone_col, date_col, "pred_arima"])

    return pd.concat(out_rows, ignore_index=True)


def prophet_forecast_per_zone(train_df, test_df, target_col,
                              zone_col="zone_id", date_col="date"):
    """
    Prophet forecasting baseline applied independently per zone.
    Captures seasonality (weekly + yearly) but ignores engineered ML features.
    """
    out_rows = []
    zones = sorted(test_df[zone_col].unique())

    for z in zones:
        tr = train_df[train_df[zone_col] == z].sort_values(date_col)
        te = test_df[test_df[zone_col] == z].sort_values(date_col)

        if len(tr) < 60 or len(te) == 0:
            continue

        # Prepare data for Prophet format
        prophet_train = tr[[date_col, target_col]].rename(
            columns={date_col: "ds", target_col: "y"}
        )

        # Prophet model with seasonal components
        m = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False
        )
        m.fit(prophet_train)

        future = pd.DataFrame({"ds": te[date_col].values})
        fc = m.predict(future)

        out_rows.append(pd.DataFrame({
            zone_col: z,
            date_col: te[date_col].values,
            "pred_prophet": fc["yhat"].values
        }))

    if not out_rows:
        return pd.DataFrame(columns=[zone_col, date_col, "pred_prophet"])

    return pd.concat(out_rows, ignore_index=True)


In [18]:

# Generate predictions from all models
pred_lag1 = eval_naive_lag1(test, target)
pred_xgb, xgb_pipe = train_xgb_forecaster(train, test, target)

# Generate classical baseline forecasts
arima_out = arima_forecast_per_zone(train, test, target)
prophet_out = prophet_forecast_per_zone(train, test, target)

# Build evaluation dataframe aligned by date and zone
eval_df = test[["date", "zone_id", target]].copy()
eval_df["pred_lag1"] = pred_lag1
eval_df["pred_xgb"] = pred_xgb

# Merge ARIMA and Prophet predictions into same table
eval_df = eval_df.merge(arima_out, on=["zone_id", "date"], how="left")
eval_df = eval_df.merge(prophet_out, on=["zone_id", "date"], how="left")

# Compute performance metrics for each model
mae_lag1, rmse_lag1 = safe_metrics(eval_df[target], eval_df["pred_lag1"])
mae_arima, rmse_arima = safe_metrics(eval_df[target], eval_df["pred_arima"])
mae_prophet, rmse_prophet = safe_metrics(eval_df[target], eval_df["pred_prophet"])
mae_xgb, rmse_xgb = safe_metrics(eval_df[target], eval_df["pred_xgb"])

# Create a single comparison table
results = pd.DataFrame({
    "Model": ["Naive Lag-1", "ARIMA", "Prophet", "XGBoost"],
    "MAE": [mae_lag1, mae_arima, mae_prophet, mae_xgb],
    "RMSE": [rmse_lag1, rmse_arima, rmse_prophet, rmse_xgb]
}).sort_values("MAE")

print(results)

# Attach zone_name if available (for readable output)
if "zone_name" in df.columns:
    zone_lookup = df[["zone_id", "zone_name"]].drop_duplicates()
    forecast_out = eval_df[["date", "zone_id"]].merge(zone_lookup, on="zone_id", how="left")
else:
    forecast_out = eval_df[["date", "zone_id"]].copy()

# Export XGBoost predictions to feed VRP routing module
forecast_out["pred_waste_tons_day"] = eval_df["pred_xgb"].values
forecast_out.to_csv("forecast_zone_day.csv", index=False)
print("Saved forecast file: forecast_zone_day.csv")

# Compute relative error (forecast accuracy vs average waste level)
mean_waste = df[target].mean()
mae_pct = (mae_xgb / mean_waste) * 100 if mean_waste else np.nan

print("Mean waste:", mean_waste)
print("Relative MAE (%):", mae_pct)


         Model       MAE      RMSE
3      XGBoost  0.801528  1.171977
2      Prophet  1.190590  1.813681
0  Naive Lag-1  1.570418  2.357977
1        ARIMA  2.037998  2.773244
Saved forecast file: forecast_zone_day.csv
Mean waste: 29.752307339324133
Relative MAE (%): 2.694001522018811
